(ch_batch_norm)=
# Batch Normalization

本节讨论如何使用 TVM 执行 batch normalization（`batch_norm`）。和 pooling 一样，`batch_norm` 也是 CNN 中常见的算子。D2L 在 [细节](https://d2l.ai/chapter_convolutional-modern/batch-norm.html) 中引入了这个算子。

从计算的角度来看，对于给定的值，`batch_norm` 减去其中的 `mean`，然后用 `variance` 的平方根除以它，与常规 normalization 没有区别。它被称为 `batch_norm`，因为均值和方差是在执行训练时从批次中获得的。在此之后，  `batch_norm` 也对该值应用仿射变换，即将其与 scale 值 $\gamma$ 相乘，然后加上 shift 值 $\beta$。由训练梯度计算得到 $\gamma$ 和 $\beta$。最后，加上一个小的正值 $\epsilon$，以防止除数为 0。

在推理的情况下，均值和方差都是确定的，因此 `batch_norm` 的过程只是几个简单的元素运算的组合。

In [1]:
import tvm
from tvm import te
from tvm_book.contrib import d2ltvm
import numpy as np

## 计算定义

在实践中，不打算执行一个值的 `batch_norm`。相反，`batch_norm` 将在卷积的输出上执行，即 (channel, height, weight) 中的 3-D 数据。

$$out[i,:,:] = \frac{data[i,:,:] - mean[i]}{\sqrt{var[i]+\epsilon}} \
* \gamma[i] + \beta[i] $$

在模型训练过程中，并根据输入 $data$ 进行计算 $mean$ 和 $var$。然而，这里重点关注模型推理，并给出了 $mean$ 和 $var$；因此不需要从中计算它们。

我们将定义这个公式的计算。实际上，`batch_norm` 是一些简单的广播和元素相关的计算的组合。注意，在 {ref}`ch_bcast_add` 中，定义了有限的 `broadcast_add`，只对二维张量执行广播加法。如果将其推广到更多维和更多的 calculator，就可以重用它们来组成 `batch_norm` 算子。这就是 TVM 所做的。

这里，为了简单起见，使用 TVM 基本算子进行广播计算。TVM 算子定义在 `TOPI` 中，TOPI 代表张量算子清单。它遵循 NumPy 约定来覆盖算术算子（即 `+`、`-`、`*`、`/`）。元素的平方根也可以在 `TOPI` 中找到。

定义 `batch_norm` 的代码片段如下所示。

In [2]:
# Save to the d2ltvm package.
from tvm_book.contrib.d2ltvm import topi

def batch_norm(c, n, eps=1e-5):
    """batch normalization
    
    c : channels
    N : input width and height
    eps : small positive value to prevent divide 0
    """
        
    X = te.placeholder((c, n, n), name='X')
    Mean = te.placeholder((c, 1, 1), name='Mean')
    Var = te.placeholder((c, 1, 1), name='Var')
    Gamma = te.placeholder((c, 1, 1), name='Gamma')
    Beta = te.placeholder((c, 1, 1), name='Beta')
    C1 = X - Mean
    C2 = topi.sqrt(Var + eps)
    Y = C1 / C2 * Gamma + Beta
    return X, Mean, Var, Gamma, Beta, Y

然后可以打印并编译 IR。IR 包括几个阶段，但应该很容易遵循。

In [3]:
c = 32
n = 28
X, Mean, Var, Gamma, Beta, Y = batch_norm(c, n)

sch = te.create_schedule(Y.op)
m = tvm.lower(sch, [X, Mean, Var, Gamma, Beta, Y], simple_mode=True)
mod = tvm.build(m)

m["main"]

PrimFunc([X, Mean, Var, Gamma, Beta, T_add]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  allocate T_subtract[float32 * 25088], storage_scope = global
  allocate T_add[float32 * 32], storage_scope = global
  for (ax0, 0, 32) {
    for (ax1, 0, 28) {
      for (ax2, 0, 28) {
        let cse_var_1 = (((ax0*784) + (ax1*28)) + ax2)
        T_subtract[cse_var_1] = (X[cse_var_1] - Mean[ax0])
      }
    }
  }
  for (ax0, 0, 32) {
    T_add[ax0] = (Var[ax0] + 1e-05f)
  }
  for (i0, 0, 32) {
    T_add[i0] = tir.sqrt(T_add[i0])
  }
  for (ax0, 0, 32) {
    for (ax1, 0, 28) {
      for (ax2, 0, 28) {
        let cse_var_2 = (((ax0*784) + (ax1*28)) + ax2)
        T_subtract[cse_var_2] = (T_subtract[cse_var_2]/T_add[ax0])
      }
    }
  }
  for (ax0, 0, 32) {
    for (ax1, 0, 28) {
      for (ax2, 0, 28) {
        let cse_var_3 = (((ax0*784) + (ax1*28)) + ax2)
        T_subtract[cse_var_3] = (T_subtract[cse_var_3]*Gamma[ax0])
      }
    }
  }
  

要执行它，需要为 `batch_norm` 创建数据。与前面获取 conv 和 pooling 数据的部分类似，定义了 `get_bn_data` 方法来生成 `batch_norm` 的数据。棘手的问题是方差必须是非负数。因此，将随机数生成器正态分布的均值移至 1（默认均值为 0，标准差为 1），得到生成结果的绝对数量。

在获得数据之后，可以简单地调用编译后的模块来执行。

In [4]:
# Save to the d2ltvm package.
def get_bn_data(c, n, constructor=None):
    """Return the batch norm data, mean, variance, gamma and beta tensors.
       Also return the empty tensor for output.

    c : channels
    n : input width and height
    constructor : user-defined tensor constructor
    """
    np.random.seed(0)
    data = np.random.normal(size=(c, n, n)).astype('float32')
    mean = np.random.normal(size=(c, 1, 1)).astype('float32')
    # move the mean of the normal distribution to be 1
    var = np.random.normal(loc=1.0, size=(c, 1, 1)).astype('float32')
    # make sure all variance numbers are not negative
    var = np.absolute(var)
    gamma = np.random.normal(size=(c, 1, 1)).astype('float32')
    beta = np.random.normal(size=(c, 1, 1)).astype('float32')
    out = np.empty((c, n, n), dtype='float32')
    if constructor:
        data, mean, var, gamma, beta, out = \
        (constructor(x) for x in [data, mean, var, gamma, beta, out])
    return data, mean, var, gamma, beta, out

data, mean, var, gamma, beta, out = get_bn_data(c, n, tvm.nd.array)
mod(data, mean, var, gamma, beta, out)

## MXNet Baseline

使用 MXNet 的 `batch_norm` 函数作为基准来检查编译函数的正确性。

MXNet 中的这个函数被定义为通用函数，用于训练和推理。在这里讨论的推理案例中，需要正确地设置相应的输入参数。一个是 `use_global_stats`，需要设置为 `True` ，因为将使用 `batch_norm` 的输入平均值和方差来计算，而不是从输入数据计算它们（训练将这样做）。另一个是 `fix_gamma`，它需要设置为 `False`，这样输入的 $\gamma$ 就会被使用，而不是将 $\gamma$ 为全设置为 1。

最后，就像在其他例子中讨论的那样，MXNet `batch_norm` 的输入数据是 4D，其中 batch 是最外维。因此，将在数据中相应地扩展这个维度。

In [5]:
import mxnet as mx

# Save to the d2ltvm package.
def get_bn_data_mxnet(c, n, ctx='cpu'):
    ctx = getattr(mx, ctx)()
    data, mean, var, gamma, beta, out = get_bn_data(c, n,
                                      lambda x: mx.nd.array(x, ctx=ctx))
    data, out = data.expand_dims(axis=0), out.expand_dims(axis=0)
    return data, mean, var, gamma, beta, out

# Save to the d2ltvm package.
def batch_norm_mxnet(data, mean, var, gamma, beta, out, eps=1e-5):
    # use_global_stats=True to use the input mean and var instead of computing
    # the mean and var of the input data.
    # fix_gamma=False so that gamma won't be set to 1.
    mx.nd.BatchNorm(data, gamma, beta, mean, var, eps, 
                    use_global_stats=True, fix_gamma=False, out=out)

data, mean, var, gamma, beta, out_mx = get_bn_data_mxnet(c, n)
batch_norm_mxnet(data, mean, var, gamma, beta, out_mx)

最后，检查结果是否与 MXNet 产生的结果足够接近。

In [6]:
np.testing.assert_allclose(out_mx[0].asnumpy(), out.asnumpy(), atol=1e-5)

## 小结

- 从计算的角度来看，`batch_norm` 是一系列广播和元素算子的组合，可以很容易地从 TVM 的张量算子库（Tensor OPerator Inventory，简称 TOPI）中得到。
- 在推理中，`batch_norm` 的 $mean$ 和 $var$ 是预定义的。